In [1]:
## Project: CommentToxicity with BERT
# ----------------------------------
# Structure:
# ├─ jigsaw-toxic-comment-classification-challenge/
# │   ├─ train.csv
# │   ├─ test.csv
# │   ├─ test_labels.csv
# │   └─ sample_submission.csv
# ├─ Toxicity_BERT.ipynb     # Jupyter notebook
# ├─ requirements.txt
# └─ .gitignore

# -------------------------------
# requirements.txt
# -------------------------------
# tensorflow>=2.10.0
# transformers>=4.30.0
# pandas
# scikit-learn
# matplotlib


# In Toxicity_BERT.ipynb (or a .py script), use the following cells/code:



In [2]:
# 1. Imports and Setup
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast, TFBertModel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [6]:

# 2. Load Data

train_df = pd.read_csv("C:/Users/kansh/Desktop/project/comment toxicity/CommentToxicity-main/jigsaw-toxic-comment-classification-challenge/train.csv")
# For multi-label: columns 2:8
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_df[label_cols] = train_df[label_cols].fillna(0)



In [8]:
# 3. Train/Validation Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['comment_text'].values,
    train_df[label_cols].values,
    test_size=0.1,
    random_state=42
)



In [9]:
# 4. Tokenizer & Model Initialization
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
bert_encoder = TFBertModel.from_pretrained(MODEL_NAME)



model.safetensors:  14%|#4        | 62.9M/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
# 5. Tokenization Function
MAX_LEN = 128
def tokenize_texts(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors='tf'
    )



In [11]:
# 6. Prepare tf.data datasets
train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    }, train_labels
)).shuffle(10000).batch(16).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': val_encodings['input_ids'],
        'attention_mask': val_encodings['attention_mask']
    }, val_labels
)).batch(16).prefetch(tf.data.AUTOTUNE)



In [12]:
# 7. Build Model
input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')

bert_outputs = bert_encoder(input_ids, attention_mask=attention_mask)
pooled_output = bert_outputs.pooler_output  # [batch_size, hidden_size]

x = tf.keras.layers.Dropout(0.3)(pooled_output)
outputs = tf.keras.layers.Dense(len(label_cols), activation='sigmoid', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                      

In [13]:
# 8. Training
EPOCHS = 3
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)




Epoch 1/3


  12/8976 [..............................] - ETA: 230:58:30 - loss: 0.4691 - accuracy: 0.1094


KeyboardInterrupt



In [ ]:
# 9. Plot Training Curves
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss curves')
plt.show()

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title('Accuracy curves')
plt.show()


In [ ]:
# 10. Save Model
model.save('toxicity_bert_model')

# 11. Inference Helper

def predict_toxicity(texts):
    enc = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='tf')
    preds = model.predict({
        'input_ids': enc['input_ids'],
        'attention_mask': enc['attention_mask']
    })
    return pd.DataFrame(preds, columns=label_cols)

# Example usage:
# results = predict_toxicity(["You are awesome!", "I hate you!"])
# print(results)